In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import scipy as sc
import time
from functools import reduce

import tensorflow as tf
from tensorflow import keras

import rl_env

In [ ]:
def create_environment(game_type='Hanabi-Full', num_players=2):
  """Creates the Hanabi environment.

  Args:
    game_type: Type of game to play. Currently the following are supported:
      Hanabi-Full: Regular game.
      Hanabi-Small: The small version of Hanabi, with 2 cards and 2 colours.
    num_players: Int, number of players to play this game.

  Returns:
    A Hanabi environment.
  """
  return rl_env.make(
      environment_name=game_type, num_players=num_players, pyhanabi_path=None)

In [ ]:
def create_actor_model(num_actions, state_shape, num_units=512):
    model = keras.Sequential([
        keras.layers.Dense(num_units, input_dim=state_shape, activation=tf.nn.relu),
        keras.layers.Dense(num_units, activation=tf.nn.relu), # needed -> guess not
        #keras.layers.Reshape((1, num_units)),
        #keras.layers.LSTM(num_units, return_sequences=True),
        #keras.layers.LSTM(num_units),
        keras.layers.Dense(num_actions, activation=tf.nn.softmax)
    ])
    return model

def create_actor_model_NEW(num_actions, state_shape, num_units=512):
    x = tf.placeholder(tf.float32, shape=[None,state_shape], name='x')
    actions = tf.placeholder(tf.float32, shape=[None,num_actions], name='actions')
    
    h1 = tf.layers.dense(x, units=num_units, activation=tf.nn.relu)
    h2 = tf.layers.dense(h1, units=num_units, activation=tf.nn.relu)
    logits = tf.layers.dense(h2, units=num_actions, activation=None)
    y = tf.nn.softmax(logits)
    
    negative_log_prob = tf.nn.softmax_cross_entropy_with_logits_v2(labels=actions, logits=logits)
    
    advantage = tf.placeholder(tf.float32, shape=[None,], name='advantage')
    
    loss = tf.reduce_mean(tf.multiply(negative_log_prob, advantage))
    optimizer = tf.train.AdamOptimizer()
    train = optimizer.minimize(loss)
    
    return y, x, train, loss, advantage, actions

def create_critic_model(state_shape, num_units=256):
    model = keras.Sequential([
        keras.layers.Dense(num_units, input_dim=state_shape, activation=tf.nn.relu),
        keras.layers.Dense(num_units, activation=tf.nn.relu),
        keras.layers.Dense(1)
    ])
    model.compile(loss='mean_squared_error',
                optimizer='adam',
                metrics=['mean_squared_error'])
    return model

In [ ]:
def get_action(actor, state, legal_moves):
    policy = actor.predict(np.reshape(state, (1,-1))).squeeze()
    # somehow only consider the legal actions
    policy_legal = np.full(policy.shape, -np.inf)
    policy_legal[legal_moves] = policy[legal_moves]
    policy_legal = sc.special.softmax(policy_legal)
    
    action = np.random.choice(policy_legal.shape[0], p=policy_legal)
    logprob = np.log(policy_legal[action])
    
    return action, logprob

def get_action_NEW(sess, y, x, state, legal_moves):
    policy = sess.run(y, {x: [state]}).squeeze()
    # somehow only consider the legal actions
    policy_legal = np.full(policy.shape, -np.inf)
    policy_legal[legal_moves] = policy[legal_moves]
    policy_legal = sc.special.softmax(policy_legal)
    
    action = np.random.choice(policy_legal.shape[0], p=policy_legal)
    logprob = np.log(policy_legal[action])
    
    return action, logprob

def get_value_estimate(critic, state):
    return critic.predict(np.reshape(state, (1,-1))).squeeze()
    

In [ ]:
def updateCritic(critic, states, Rts, ep_lens):
    train_data = []
    train_label = []
    
    for traj in range(states.shape[0]):
        H = ep_lens[traj]
        train_data.extend(states[traj, :H])
        train_label.extend(Rts[traj, :H])
    
    critic.train_on_batch(np.array(train_data), train_label)
    
def updateCritic_NEW(critic, states, Rts):
    train_data = []
    train_label = []
    
    for traj in range(states.shape[0]):
        H = ep_lens[traj]
        train_data.extend(states[traj, :H])
        train_label.extend(Rts[traj, :H])
    
    critic.train_on_batch(np.array(train_data), train_label)
    
    
def update_actor(train, x, states):
    sess.run((train, loss), {x: states})
    
    return train

In [ ]:
def calculateRtsAndAts(rewards, baselines, ep_lens, gamma=0.95):
    Rts = np.zeros(rewards.shape)
    Ats = np.zeros(rewards.shape)
    
    for traj in range(rewards.shape[0]):
        H = ep_lens[traj]
        Rt = 0
        for t in reversed(range(H)):
            Rt += rewards[traj, t]
            Rts[traj, t] = Rt
            At = Rt -  baselines[traj, t]
            Ats[traj, t] = At
            
            Rt *= gamma
            
    return Rts, Ats

def calculateRtsAndNormalizedAts(rewards, baselines, gamma=0.95):
    Rts = []
    Ats = []
    
    for traj in range(len(rewards)):
        Rt = 0
        T = len(rewards[traj])
        Rts.append(np.zeros(T))
        Ats.append(np.zeros(T))
        for t in reversed(range(T - 1)):
            Rt += rewards[traj][t]
            Rts[traj][t] = Rt
            At = Rt - baselines[traj][t]
            Ats[traj][t] = At
            
            Rt *= gamma
        
        #normalize only advantages,since rewards are used for training the critic
        Ats[traj] -= Ats[traj].mean()
        Ats[traj] /= (Ats[traj].std() + 10**(-10))
            
    return Rts, Ats

# Old O(n^2) version
def calculateRtsAndAts2(rewards, baselines, ep_lens, gamma=0.95):
    Rts = np.zeros(rewards.shape)
    Ats = np.zeros(rewards.shape)
    
    for traj in range(rewards.shape[0]):
        H = ep_lens[traj]
        for t in range(H):
            Rt = calculateSingleRt(rewards[traj, t:H], gamma)
            At = Rt -  baselines[traj, t]
            Rts[traj, t] = Rt
            Ats[traj, t] = At
            
    return Rts, Ats

def calculateSingleRt(rewards, gamma):
    rt = 0
    for t in range(len(rewards)):
        rt += gamma**(t)*rewards[t]
    return rt

In [ ]:
actor = create_actor_model(env.num_moves(), env.vectorized_observation_shape()[0])
critic = create_critic_model(env.vectorized_observation_shape()[0])
actor.summary()
critic.summary()

In [ ]:
num_iterations = 1
num_trajectories = 1000
t0 = time.time()


for it in range(num_iterations):
    rewards = np.zeros((num_trajectories, 110))
    ep_lens = np.zeros(num_trajectories).astype(int)


    # collect trajectories
    for traj in range(num_trajectories):
        # reset environment
        observations = env.reset()
        reward = 0
        is_done = False

        # collect one trajectory
        for st in range(110):
            # extract legal moves and state from observations
            moves = observations['player_observations'][st % 2]['legal_moves_as_int']
            state = observations['player_observations'][st % 2]['vectorized']

            # get next action
            action, _ = get_action_NEW(sess, y, x, state, moves)

            # do next step
            observations, reward, is_done, _ = env.step(action)

            # store reward after executing action
            rewards[traj, st] = reward


            if (is_done or st == 110-1):
                ep_lens[traj] = st
                #print("Done", ep, st)
                #print(observations['player_observations'][0]['pyhanabi'])
                #print('reward', reward)
                #print(' --- ')
                break

    
t1 = time.time()
print(rewards.shape, ep_lens.shape, t1-t0)

In [ ]:
num_iterations = 100*200
num_trajectories = 10
t0 = time.time()

# init actor neural net (a custom tf graph)
y, x, train, loss, advantage, actions = create_actor_model_NEW(env.num_moves(), env.vectorized_observation_shape()[0])
init = tf.global_variables_initializer()

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

for it in range(num_iterations):
    actions_ = np.zeros((num_trajectories, 110, env.num_moves())) 
    states = np.zeros((num_trajectories, 110, env.vectorized_observation_shape()[0]))
    rewards = np.zeros((num_trajectories, 110))
    baselines = np.zeros((num_trajectories, 110))
    logprobs = np.zeros((num_trajectories, 110))
    ep_lens = np.zeros(num_trajectories).astype(int)


    # collect trajectories
    for traj in range(num_trajectories):
        # reset environment
        observations = env.reset()
        reward = 0
        is_done = False

        # collect one trajectory
        for st in range(110):
            # extract legal moves and state from observations
            moves = observations['player_observations'][st % 2]['legal_moves_as_int']
            state = observations['player_observations'][st % 2]['vectorized']

            # get next action
            action, logprob = get_action_NEW(sess, y, x, state, moves)

            # store variables
            actions_[traj, st, action] = 1
            states[traj, st] = state
            logprobs[traj, st] = logprob

            # get baseline(value estimate)
            baselines[traj, st] = get_value_estimate(critic, state)

            # do next step
            observations, reward, is_done, _ = env.step(action)

            # store reward after executing action
            rewards[traj, st] = reward


            if (is_done or st == 110-1):
                ep_lens[traj] = st
                #print("Done", ep, st)
                #print(observations['player_observations'][0]['pyhanabi'])
                #print('reward', reward)
                #print(' --- ')
                break

    # collected a bunch of trajectories, now calculate discounted rewards (Rts)
    # TODO update critic net and actor net
    # TODO normalize Rts and Ats after calculating them or before? 
    Rts, Ats = calculateRtsAndAts(rewards, baselines, ep_lens)
    discounted_Rts, discounted_Ats = (Rts-Rts.mean())/Rts.std(), (Ats-Ats.mean())/Ats.std()
    x_train = np.reshape(states, (-1, env.vectorized_observation_shape()[0]))
    adv = discounted_Ats.reshape((-1))
    acts = actions_.reshape((-1,env.num_moves()))
    updateCritic(critic, states, Rts, ep_lens)
    sess.run((train, loss), {x: x_train, advantage: adv, actions: acts})
    
    
t1 = time.time()
print(rewards.shape, ep_lens.shape, t1-t0)

In [ ]:
num_iterations = 100
num_trajectories = 100
t0 = time.time()

# create hanabi environment
env = create_environment()
# init simple keras critic 
critic = create_critic_model(env.vectorized_observation_shape()[0])
# init actor neural net (a custom tf graph)
y, x, train, loss, advantage, actions = create_actor_model_NEW(env.num_moves(), env.vectorized_observation_shape()[0])
init = tf.global_variables_initializer()

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

losses = []
for it in range(num_iterations):
    actions_ = [] 
    states = []
    rewards = []
    baselines = []
    logprobs = []
    ep_lens = []


    # collect trajectories
    for traj in range(num_trajectories):
        # reset environment
        observations = env.reset()
        reward = 0
        is_done = False
        actions_.append([])
        states.append([])
        rewards.append([])
        baselines.append([])
        logprobs.append([])

        # collect one trajectory
        for st in range(110):
            # extract legal moves and state from observations
            moves = observations['player_observations'][st % 2]['legal_moves_as_int']
            state = observations['player_observations'][st % 2]['vectorized']

            # get next action
            action, logprob = get_action_NEW(sess, y, x, state, moves)

            # store variables
            action_ = np.zeros(env.num_moves())
            action_[action] = 1
            actions_[traj].append(action_)
            states[traj].append(state)
            logprobs[traj].append(logprob)

            # get baseline(value estimate)
            baselines[traj].append(get_value_estimate(critic, state))

            # do next step
            observations, reward, is_done, _ = env.step(action)

            # store reward after executing action
            rewards[traj].append(reward)


            if (is_done or st == 110-1):
                ep_lens.append(st)
                #print("Done", ep, st)
                #print(observations['player_observations'][0]['pyhanabi'])
                #print('reward', reward)
                #print(' --- ')
                break

    # collected a bunch of trajectories, now calculate discounted rewards (Rts)
    Rts, Ats = calculateRtsAndNormalizedAts(rewards, baselines)
    # 'flatten' the lists
    x_train = np.array(reduce(lambda x,y: np.vstack((x,y)), states))
    adv = np.array(reduce(lambda x,y: np.hstack((x,y)), Ats))
    rew = np.array(reduce(lambda x,y: np.hstack((x,y)), Rts))
    acts = np.array(reduce(lambda x,y: np.vstack((x,y)), actions_))
    
    # train critic and actor
    critic.train_on_batch(x_train, rew)
    _, loss_t = sess.run((train, loss), {x: x_train, advantage: adv, actions: acts})
    losses.append(loss_t)
    
    
t1 = time.time()
print(len(rewards), len(ep_lens), len(losses), t1-t0)

In [ ]:
losses

In [ ]:
plt.plot(list(map(lambda x: -x,losses)))
plt.show()